In [ ]:
from myria import *
import numpy
from raco.backends.myria.connection import MyriaConnection
from raco.backends.myria.catalog import MyriaCatalog
from raco.backends.myria import MyriaLeftDeepTreeAlgebra
from raco.compile import optimize

import raco.myrial.interpreter as interpreter
import raco.myrial.parser as myrialparser

from graphviz import Digraph

def graph_to_dot_object(graph, **kwargs):
    """Graph is expected to be a dict of the form { 'nodes' : list(), 'edges' :
    list() }. This function returns a string that will be input to dot."""

    title = kwargs.get('title', '')
    dot = Digraph(comment = title)
    for n in graph['nodes']:
        dot.node(str(id(n)), n.shortStr().replace(r'"', r'\"'))
    for (x,y) in graph['edges']:
        dot.edge(str(id(x)), str(id(y)))

    return dot

def operator_to_dot_object(operator, graph=None, **kwargs):
    """Operator is expected to be an object of class raco.algebra.Operator"""
    graph = operator.collectGraph(graph)
    return graph_to_dot_object(graph, **kwargs)

In [ ]:
myrial_code = """
T1 = scan(TwitterK);
T2 = scan(TwitterK);
Joined = [from T1, T2
          where T1.$1 = T2.$0 and T1.$1 > 100
          emit T1.$0 as src, T1.$1 as link, T2.$1 as dst];
store(Joined, TwoHopsInTwitter);
"""

# Parser

In [ ]:
parser = myrialparser.Parser()
statement_list = parser.parse(myrial_code)
statement_list

# Statement Processor

In [ ]:
conn = MyriaConnection(rest_url='http://localhost:8753', execution_url='http://localhost:8080')
myriacatalog = MyriaCatalog(conn)
processor = interpreter.StatementProcessor(myriacatalog, True)
processor.evaluate(statement_list)

# Optimizer

## Logical Plan

In [ ]:
logical = processor.get_logical_plan()
operator_to_dot_object(logical)

## Physical Plan

In [ ]:
physical_plan = processor.get_physical_plan(target_alg=MyriaLeftDeepTreeAlgebra())
operator_to_dot_object(physical_plan)

## Submit query to the execution engine

In [ ]:
from raco.backends.myria import compile_to_json
json_plan = compile_to_json(str(myrial_code), logical, physical_plan)

In [ ]:
conn.submit_query(json_plan)

## Rules

In [ ]:
MyriaLeftDeepTreeAlgebra().opt_rules()

## Another example

In [ ]:
myrial_code = """
E = scan(TwitterK);
V = select distinct E.$0 from E;
CC = [from V emit V.$0 as node_id, V.$0 as component_id];
do
  new_CC = [from E, CC where E.$0 = CC.$0 emit E.$1, CC.$1] + CC;
  new_CC = [from new_CC emit new_CC.$0, MIN(new_CC.$1)];
  delta = diff(CC, new_CC);
  CC = new_CC;
while [from delta emit count(*) > 0];
comp = [from CC emit CC.$1 as id, count(CC.$0) as cnt];
store(comp, TwitterCC);
"""

In [ ]:
statement_list = parser.parse(myrial_code)
conn = MyriaConnection(rest_url='http://localhost:8753', execution_url='http://localhost:8080')
myriacatalog = MyriaCatalog(conn)
processor = interpreter.StatementProcessor(myriacatalog, True)
processor.evaluate(statement_list)
logical = processor.get_logical_plan()
physical_plan = processor.get_physical_plan(target_alg=MyriaLeftDeepTreeAlgebra())

In [ ]:
operator_to_dot_object(logical)

In [ ]:
operator_to_dot_object(physical_plan)

In [ ]:
json_plan = compile_to_json(str(myrial_code), logical, physical_plan)
conn.submit_query(json_plan)